In [14]:
from openai import OpenAI
from pinecone import Pinecone
from dotenv import load_dotenv
import os
load_dotenv()

True

In [24]:
from openai import OpenAI
from pinecone import Pinecone
from dotenv import load_dotenv
import os
load_dotenv()

openai_client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_API_BASE")
)

user_question = "what is possible with embroidery in HCI textiles research?"

# use OpenAI API text-embedding-3-small to get embeddings for a question
response = openai_client.embeddings.create(
    input=user_question,
    model="text-embedding-3-small" 
)
question_vector = response.data[0].embedding

# use Pinecone to search for the most relevant papers
pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
index_name = "textiles-hci-01" 
index = pc.Index(index_name)

# search for the most relevant papers
# index.query(vector=[0.1, 0.2, 0.3], top_k=10, namespace='my_namespace')
query_results = index.query(
    top_k=5, # top 5 most relevant papers
    vector=question_vector, 
    includeValues=True,
    includeMetadata=True,
    namespace="pdf"
    # filter: { genre: { '$eq': 'action' }}
    )  

# get the metadata of the most relevant papers
# print(query_results)
# contexts = [item['metadata']['abstract'] for item in query_results['matches']]
contexts = [item['metadata']['text'] for item in query_results['matches']]
dois = [f"[https://doi.org/10.1145/{item['metadata']['doi']}](https://doi.org/10.1145/{item['metadata']['doi']})" for item in query_results['matches']]
print(dois)

# combine the contexts and the question
context_combined = "\n".join(contexts)
chat_input = f"Context: {context_combined}\nQuestion: {user_question}"

response = openai_client.chat.completions.create(
  model="gpt-3.5-turbo", 
  messages=[
    {"role": "system", "content": "You are a helpful research assistant and an expert in textile research and HCI research."},
    {"role": "user", "content": chat_input}
  ],
#   prompt=chat_input,
  max_tokens=1000 # limit the response to 1000 tokens
)

# get the answer
# get the answer
answer = response.choices[0].message.content

# include the DOIs of the most relevant papers
answer = f"{answer}\n\n### DOIs of the most relevant papers:\n" + "- " + "\n- ".join(dois)

print(answer)

['[https://doi.org/10.1145/3313831.3376305](https://doi.org/10.1145/3313831.3376305)', '[https://doi.org/10.1145/2347504.2347542](https://doi.org/10.1145/2347504.2347542)', '[https://doi.org/10.1145/3563657.3596086](https://doi.org/10.1145/3563657.3596086)', '[https://doi.org/10.1145/3544548.3581156](https://doi.org/10.1145/3544548.3581156)', '[https://doi.org/10.1145/3563657.3596086](https://doi.org/10.1145/3563657.3596086)']
Embroidery in HCI textiles research offers a wide range of possibilities due to its versatility and unique properties. Some potential applications and advancements that can be explored with embroidery in HCI textiles research include:

1. **Integration of Electronics:** Embroidery can be used to seamlessly integrate electronic components such as sensors, actuators, and conductive threads into textiles. This allows for the creation of interactive garments and accessories with embedded technology for various applications including health monitoring, gesture recogni

In [6]:
import requests

# cloud function URL
url = "https://texverse-ai-qa-lebwgsie4q-uc.a.run.app"

# user question
params = {'text': 'what is possible with knitting in HCI textiles research?'}

response = requests.get(url, params=params)

print(response.text)

500 Internal Server Error: The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.


In [15]:
response.text

''

In [5]:
dois = ["https://doi.org/10.1145/3559400.3562001", "https://doi.org/10.1145/3559400.3562002", "https://doi.org/10.1145/3559400.3562003", "https://doi.org/10.1145/3559400.3562004", "https://doi.org/10.1145/3559400.3562005"]
answer = "1213"
answer = f"{answer}\n\n ### DOIs of the most relevant papers:\n" + "\n - ".join(dois)
answer

'1213\n\n ### DOIs of the most relevant papers:\nhttps://doi.org/10.1145/3559400.3562001\n - https://doi.org/10.1145/3559400.3562002\n - https://doi.org/10.1145/3559400.3562003\n - https://doi.org/10.1145/3559400.3562004\n - https://doi.org/10.1145/3559400.3562005'